In [2]:
import torch
import re

In [9]:
pth_source = torch.load(r'/data/hyogun/repos/haawron_mmaction2/data/weights/svt/releases/download/v1.0/kinetics400_vitb_ssl.pth')
pth_ref = torch.load(r'/data/hyogun/repos/haawron_mmaction2/data/weights/timesformer/timesformer_divST_8x32x1_15e_kinetics400_rgb-3f8e5d03.pth.1')

In [5]:
pth_ref.keys() 

dict_keys(['meta', 'state_dict'])

In [8]:
(pth_ref["state_dict"]['backbone.cls_token'] == pth_source['backbone.cls_token']).all() 

tensor(False)

In [ ]:
for i, (k1, k2) in enumerate(zip(pth_ref['state_dict'], pth_source)):
    print(f'{i:3d}  {k1:80} {str(tuple(pth_ref["state_dict"][k1].shape)):16s}\t{str(tuple(pth_source[k2].shape)):16s} {i:3d}  {k2:60} ')

  0  backbone.cls_token                                                               (1, 1, 768)     	(1, 1, 768)        0  backbone.cls_token                                           
  1  backbone.pos_embed                                                               (1, 197, 768)   	(1, 197, 768)      1  backbone.pos_embed                                           
  2  backbone.time_embed                                                              (1, 8, 768)     	(1, 8, 768)        2  backbone.time_embed                                          
  3  backbone.patch_embed.projection.weight                                           (768, 3, 16, 16)	(768, 3, 16, 16)   3  backbone.patch_embed.proj.weight                             
  4  backbone.patch_embed.projection.bias                                             (768,)          	(768,)             4  backbone.patch_embed.proj.bias                               
  5  backbone.norm.weight                                        

In [16]:
new_state_dict_order_and_key = []
for i, k in enumerate(pth_source.keys()):
    if 'temporal_attn' in k:
        new_state_dict_order_and_key.append((i + 2, k))
    elif 'temporal_fc' in k:
        new_state_dict_order_and_key.append((i - 4, k))
    else:
        new_state_dict_order_and_key.append((i, k))
new_state_dict_order_and_key = sorted(new_state_dict_order_and_key, key=lambda elem: elem[0])
new_state_dict_order_and_key[:20]

[(0, 'backbone.cls_token'),
 (1, 'backbone.pos_embed'),
 (2, 'backbone.time_embed'),
 (3, 'backbone.patch_embed.proj.weight'),
 (4, 'backbone.patch_embed.proj.bias'),
 (5, 'backbone.blocks.0.norm1.weight'),
 (6, 'backbone.blocks.0.norm1.bias'),
 (7, 'backbone.blocks.0.attn.qkv.weight'),
 (8, 'backbone.blocks.0.attn.qkv.bias'),
 (9, 'backbone.blocks.0.attn.proj.weight'),
 (10, 'backbone.blocks.0.attn.proj.bias'),
 (11, 'backbone.blocks.0.temporal_norm1.weight'),
 (12, 'backbone.blocks.0.temporal_norm1.bias'),
 (13, 'backbone.blocks.0.temporal_fc.weight'),
 (14, 'backbone.blocks.0.temporal_fc.bias'),
 (15, 'backbone.blocks.0.temporal_attn.qkv.weight'),
 (16, 'backbone.blocks.0.temporal_attn.qkv.bias'),
 (17, 'backbone.blocks.0.temporal_attn.proj.weight'),
 (18, 'backbone.blocks.0.temporal_attn.proj.bias'),
 (19, 'backbone.blocks.0.norm2.weight')]

In [23]:
pth_new = pth_ref.copy()
for i, (k1, (j, k2)) in enumerate(zip(pth_ref['state_dict'], new_state_dict_order_and_key)):
   assert i == j, f'{i} {j}'
   pth_new['state_dict'][k1] = pth_source[k2] 

In [24]:
path = '/data/hyogun/repos/haawron_mmaction2/data/weights/svt/releases/download/v1.0/SVT_mmaction.pth'
torch.save(pth_new, path)

In [22]:
(pth_new['backbone.cls_token'] == pth_ref['state_dict']['backbone.cls_token']).all()

tensor(False)

In [ ]:
zzzz.keys()